# Query Optimization: EXPLAIN Plans & Query Tuning

Query optimization is the process of improving query performance by analyzing execution plans, understanding how the database engine processes queries, and applying best practices to reduce resource consumption and execution time.

## Learning Objectives
- Understand query execution plans and how to read them
- Use EXPLAIN to analyze query performance
- Apply optimization techniques to improve query speed
- Identify and fix common anti-patterns

---
## 1. Understanding Query Execution Plans

A **query execution plan** (or query plan) describes how the database engine will execute a query. It shows:

| Component | Description |
|-----------|-------------|
| **Scan Type** | How tables are accessed (full scan, index scan, etc.) |
| **Join Method** | How tables are joined (nested loop, hash, merge) |
| **Cost Estimates** | Relative cost of operations |
| **Row Estimates** | Expected number of rows processed |
| **Order of Operations** | Sequence in which operations execute |

### Common Scan Types
- **SCAN TABLE**: Full table scan - reads every row (expensive for large tables)
- **SEARCH TABLE USING INDEX**: Uses an index to find rows (efficient)
- **SEARCH TABLE USING COVERING INDEX**: Index contains all needed columns (most efficient)

---
## 2. Setup: Create Sample Database

In [ ]:
import sqlite3
import random
import time
from datetime import datetime, timedelta

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables without indexes first
cursor.executescript('''
    CREATE TABLE customers (
        customer_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT UNIQUE,
        city TEXT,
        country TEXT,
        created_at TEXT
    );
    
    CREATE TABLE products (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        category TEXT,
        price REAL,
        stock_quantity INTEGER
    );
    
    CREATE TABLE orders (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        order_date TEXT,
        total_amount REAL,
        status TEXT,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    
    CREATE TABLE order_items (
        item_id INTEGER PRIMARY KEY,
        order_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        unit_price REAL,
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    );
''')

# Insert sample data
cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'London', 'Paris', 'Tokyo']
countries = ['USA', 'USA', 'USA', 'USA', 'USA', 'UK', 'France', 'Japan']
categories = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports']
statuses = ['pending', 'shipped', 'delivered', 'cancelled']

# Insert 1000 customers
for i in range(1, 1001):
    city_idx = random.randint(0, len(cities) - 1)
    cursor.execute(
        'INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?)',
        (i, f'Customer_{i}', f'customer{i}@email.com', cities[city_idx], countries[city_idx],
         (datetime.now() - timedelta(days=random.randint(1, 365))).isoformat())
    )

# Insert 100 products
for i in range(1, 101):
    cursor.execute(
        'INSERT INTO products VALUES (?, ?, ?, ?, ?)',
        (i, f'Product_{i}', random.choice(categories), round(random.uniform(10, 500), 2), random.randint(0, 1000))
    )

# Insert 5000 orders
for i in range(1, 5001):
    cursor.execute(
        'INSERT INTO orders VALUES (?, ?, ?, ?, ?)',
        (i, random.randint(1, 1000), 
         (datetime.now() - timedelta(days=random.randint(1, 180))).isoformat(),
         round(random.uniform(20, 1000), 2), random.choice(statuses))
    )

# Insert 15000 order items
for i in range(1, 15001):
    cursor.execute(
        'INSERT INTO order_items VALUES (?, ?, ?, ?, ?)',
        (i, random.randint(1, 5000), random.randint(1, 100), random.randint(1, 5), round(random.uniform(10, 500), 2))
    )

conn.commit()
print("Database created with:")
print(f"  - 1,000 customers")
print(f"  - 100 products")
print(f"  - 5,000 orders")
print(f"  - 15,000 order items")

---
## 3. Using EXPLAIN QUERY PLAN

SQLite provides `EXPLAIN QUERY PLAN` to show how a query will be executed. This is crucial for understanding performance characteristics.

In [ ]:
def explain_query(conn, query, params=None):
    """Display the query execution plan."""
    cursor = conn.cursor()
    explain_sql = f"EXPLAIN QUERY PLAN {query}"
    
    if params:
        cursor.execute(explain_sql, params)
    else:
        cursor.execute(explain_sql)
    
    print("Query:")
    print(f"  {query}")
    print("\nExecution Plan:")
    for row in cursor.fetchall():
        # Format: (id, parent, notused, detail)
        indent = "  " * row[1] if row[1] else ""
        print(f"  {indent}└── {row[3]}")
    print()

# Example 1: Simple SELECT with full table scan
explain_query(conn, "SELECT * FROM customers WHERE city = 'New York'")

In [ ]:
# Example 2: Query with PRIMARY KEY (uses automatic index)
explain_query(conn, "SELECT * FROM customers WHERE customer_id = 500")

In [ ]:
# Example 3: JOIN query without indexes
explain_query(conn, """
    SELECT c.name, o.order_date, o.total_amount
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.status = 'shipped'
""")

---
## 4. Index Optimization

Indexes dramatically improve query performance by allowing the database to find rows without scanning the entire table.

### When to Create Indexes
- Columns frequently used in `WHERE` clauses
- Columns used in `JOIN` conditions
- Columns used in `ORDER BY` or `GROUP BY`
- Columns with high selectivity (many unique values)

In [ ]:
def benchmark_query(conn, query, params=None, iterations=100):
    """Benchmark query execution time."""
    cursor = conn.cursor()
    start = time.perf_counter()
    
    for _ in range(iterations):
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        cursor.fetchall()
    
    elapsed = (time.perf_counter() - start) / iterations * 1000
    return elapsed

# Benchmark BEFORE index
query = "SELECT * FROM orders WHERE status = 'shipped' AND customer_id = ?"
time_before = benchmark_query(conn, query, (100,))
print(f"Before index: {time_before:.3f} ms per query")

# Show execution plan before index
explain_query(conn, query.replace('?', '100'))

In [ ]:
# Create indexes
cursor.executescript('''
    CREATE INDEX idx_orders_status ON orders(status);
    CREATE INDEX idx_orders_customer ON orders(customer_id);
    CREATE INDEX idx_orders_customer_status ON orders(customer_id, status);
    CREATE INDEX idx_customers_city ON customers(city);
    CREATE INDEX idx_order_items_order ON order_items(order_id);
    CREATE INDEX idx_order_items_product ON order_items(product_id);
''')

print("Indexes created!")

# Benchmark AFTER index
time_after = benchmark_query(conn, query, (100,))
print(f"\nAfter index: {time_after:.3f} ms per query")
print(f"Speedup: {time_before/time_after:.1f}x faster")

# Show execution plan after index
explain_query(conn, query.replace('?', '100'))

---
## 5. Composite Index Optimization

A **composite index** (multi-column index) is efficient when queries filter on multiple columns. Column order matters!

In [ ]:
# Query that benefits from composite index (customer_id, status)
explain_query(conn, """
    SELECT * FROM orders 
    WHERE customer_id = 100 AND status = 'shipped'
""")

# Query that uses only the first column of composite index
explain_query(conn, """
    SELECT * FROM orders 
    WHERE customer_id = 100
""")

# Query on second column only - may not use composite index efficiently
explain_query(conn, """
    SELECT * FROM orders 
    WHERE status = 'shipped'
""")

---
## 6. Covering Indexes

A **covering index** contains all columns needed by the query, avoiding the need to access the table data ("index-only scan").

In [ ]:
# Create a covering index for a common query pattern
cursor.execute('''
    CREATE INDEX idx_orders_covering ON orders(status, order_date, total_amount)
''')

# This query can be satisfied entirely from the index
explain_query(conn, """
    SELECT status, order_date, total_amount 
    FROM orders 
    WHERE status = 'shipped'
    ORDER BY order_date
""")

---
## 7. Common Anti-Patterns and Fixes

### Anti-Pattern 1: Using Functions on Indexed Columns

Applying functions to indexed columns prevents index usage.

In [ ]:
# ❌ BAD: Function on indexed column prevents index usage
print("❌ BAD: Using UPPER() on indexed column")
explain_query(conn, """
    SELECT * FROM customers 
    WHERE UPPER(city) = 'NEW YORK'
""")

# ✅ GOOD: Compare with consistent case or use expression index
print("✅ GOOD: Store data in consistent case, query directly")
explain_query(conn, """
    SELECT * FROM customers 
    WHERE city = 'New York'
""")

### Anti-Pattern 2: SELECT *

Selecting all columns when only a few are needed wastes resources and prevents covering index usage.

In [ ]:
# ❌ BAD: SELECT * fetches unnecessary columns
print("❌ BAD: SELECT * when only name is needed")
bad_query = "SELECT * FROM customers WHERE city = 'New York'"
time_bad = benchmark_query(conn, bad_query)
print(f"   Time: {time_bad:.3f} ms")

# ✅ GOOD: Select only needed columns
print("\n✅ GOOD: SELECT only required columns")
good_query = "SELECT name, email FROM customers WHERE city = 'New York'"
time_good = benchmark_query(conn, good_query)
print(f"   Time: {time_good:.3f} ms")
print(f"   Improvement: {((time_bad - time_good) / time_bad * 100):.1f}% faster")

### Anti-Pattern 3: N+1 Query Problem

Executing one query to get a list, then N queries to get related data.

In [ ]:
# ❌ BAD: N+1 Query Pattern
print("❌ BAD: N+1 Query Pattern")
start = time.perf_counter()

# First query: get all orders
cursor.execute("SELECT order_id, customer_id FROM orders LIMIT 100")
orders = cursor.fetchall()

# N queries: get customer for each order
results = []
for order_id, customer_id in orders:
    cursor.execute("SELECT name FROM customers WHERE customer_id = ?", (customer_id,))
    customer = cursor.fetchone()
    results.append((order_id, customer[0] if customer else None))

time_n_plus_1 = (time.perf_counter() - start) * 1000
print(f"   Time: {time_n_plus_1:.3f} ms (101 queries)")

# ✅ GOOD: Single JOIN query
print("\n✅ GOOD: Single JOIN Query")
start = time.perf_counter()

cursor.execute("""
    SELECT o.order_id, c.name 
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    LIMIT 100
""")
results = cursor.fetchall()

time_join = (time.perf_counter() - start) * 1000
print(f"   Time: {time_join:.3f} ms (1 query)")
print(f"   Speedup: {time_n_plus_1/time_join:.1f}x faster")

### Anti-Pattern 4: OR Conditions on Different Columns

OR conditions can prevent index usage. Use UNION or restructure queries.

In [ ]:
# ❌ BAD: OR across different columns
print("❌ BAD: OR condition across different columns")
explain_query(conn, """
    SELECT * FROM customers 
    WHERE city = 'New York' OR email LIKE '%@gmail.com'
""")

# ✅ GOOD: Use UNION to leverage indexes on each column
print("✅ GOOD: UNION allows each query to use its index")
explain_query(conn, """
    SELECT * FROM customers WHERE city = 'New York'
    UNION
    SELECT * FROM customers WHERE email LIKE '%@gmail.com'
""")

### Anti-Pattern 5: LIKE with Leading Wildcard

In [ ]:
# ❌ BAD: Leading wildcard prevents index usage
print("❌ BAD: LIKE with leading wildcard")
explain_query(conn, """
    SELECT * FROM customers 
    WHERE email LIKE '%@gmail.com'
""")

# ✅ GOOD: Trailing wildcard can use index
print("✅ GOOD: LIKE with trailing wildcard")
explain_query(conn, """
    SELECT * FROM customers 
    WHERE email LIKE 'customer1%'
""")

### Anti-Pattern 6: Inefficient Subqueries

In [ ]:
# ❌ BAD: Correlated subquery (runs for each row)
print("❌ BAD: Correlated subquery")
explain_query(conn, """
    SELECT c.name,
           (SELECT COUNT(*) FROM orders o WHERE o.customer_id = c.customer_id) as order_count
    FROM customers c
    WHERE c.city = 'New York'
""")

# ✅ GOOD: Use JOIN with GROUP BY
print("✅ GOOD: JOIN with GROUP BY")
explain_query(conn, """
    SELECT c.name, COUNT(o.order_id) as order_count
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    WHERE c.city = 'New York'
    GROUP BY c.customer_id, c.name
""")

---
## 8. Query Optimization Checklist

Use this checklist when optimizing queries:

In [ ]:
def analyze_query_optimization(conn, query):
    """Analyze a query and provide optimization suggestions."""
    cursor = conn.cursor()
    
    print("=" * 60)
    print("QUERY OPTIMIZATION ANALYSIS")
    print("=" * 60)
    print(f"\nQuery: {query[:100]}..." if len(query) > 100 else f"\nQuery: {query}")
    
    # Get execution plan
    cursor.execute(f"EXPLAIN QUERY PLAN {query}")
    plan = cursor.fetchall()
    
    print("\n📋 Execution Plan:")
    issues = []
    for row in plan:
        detail = row[3]
        print(f"   {detail}")
        
        # Check for potential issues
        if 'SCAN TABLE' in detail and 'USING' not in detail:
            issues.append(f"⚠️  Full table scan detected: {detail}")
        if 'USING TEMPORARY' in detail:
            issues.append("⚠️  Temporary table being used")
        if 'USING FILESORT' in detail:
            issues.append("⚠️  File sorting required")
    
    # Benchmark
    time_ms = benchmark_query(conn, query, iterations=50)
    print(f"\n⏱️  Average execution time: {time_ms:.3f} ms")
    
    # Report issues
    if issues:
        print("\n🔍 Potential Issues:")
        for issue in issues:
            print(f"   {issue}")
    else:
        print("\n✅ No obvious issues detected")
    
    print("\n" + "=" * 60)

# Analyze a complex query
analyze_query_optimization(conn, """
    SELECT c.name, c.city, COUNT(o.order_id) as total_orders, SUM(o.total_amount) as total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    WHERE o.status = 'delivered'
    GROUP BY c.customer_id
    HAVING SUM(o.total_amount) > 500
    ORDER BY total_spent DESC
    LIMIT 10
""")

---
## 9. Advanced: Query Statistics with EXPLAIN

In [ ]:
def detailed_explain(conn, query):
    """Show detailed EXPLAIN output (bytecode-level)."""
    cursor = conn.cursor()
    cursor.execute(f"EXPLAIN {query}")
    
    print("Detailed EXPLAIN (Virtual Machine Operations):")
    print("-" * 70)
    print(f"{'addr':>4} | {'opcode':<15} | {'p1':>5} | {'p2':>5} | {'p3':>5} | comment")
    print("-" * 70)
    
    for row in cursor.fetchall()[:15]:  # Show first 15 operations
        addr, opcode, p1, p2, p3, p4, p5, comment = row
        print(f"{addr:>4} | {opcode:<15} | {p1:>5} | {p2:>5} | {p3:>5} | {comment or ''}")
    print("...")

detailed_explain(conn, "SELECT * FROM orders WHERE status = 'shipped' LIMIT 5")

---
## 10. Index Maintenance and Statistics

In [ ]:
# List all indexes in the database
cursor.execute("""
    SELECT name, tbl_name, sql 
    FROM sqlite_master 
    WHERE type = 'index' AND sql IS NOT NULL
    ORDER BY tbl_name
""")

print("📊 Database Indexes:")
print("=" * 60)
current_table = None
for name, table, sql in cursor.fetchall():
    if table != current_table:
        print(f"\n📁 Table: {table}")
        current_table = table
    # Extract column info from SQL
    print(f"   └── {name}")

# Run ANALYZE to update query planner statistics
print("\n\n🔄 Running ANALYZE to update statistics...")
cursor.execute("ANALYZE")
print("✅ Statistics updated!")

---
## 🎯 Key Takeaways

### Query Execution Plans
- Always use `EXPLAIN QUERY PLAN` to understand how queries execute
- Look for **SCAN TABLE** without indexes (expensive full scans)
- Prefer **SEARCH TABLE USING INDEX** or **COVERING INDEX**

### Index Best Practices
| Do ✅ | Don't ❌ |
|-------|----------|
| Index columns in WHERE, JOIN, ORDER BY | Over-index (slows writes) |
| Use composite indexes for multi-column filters | Ignore column order in composite indexes |
| Consider covering indexes for frequent queries | Index low-selectivity columns alone |
| Run ANALYZE after bulk data changes | Forget to maintain index statistics |

### Common Anti-Patterns to Avoid
1. **Functions on indexed columns** - Prevents index usage
2. **SELECT *** - Fetches unnecessary data
3. **N+1 queries** - Use JOINs instead of loops
4. **Leading wildcards** - `LIKE '%value'` can't use indexes
5. **Correlated subqueries** - Rewrite as JOINs
6. **Missing indexes on foreign keys** - Slows JOINs

### Optimization Process
1. **Measure** - Benchmark before optimizing
2. **Analyze** - Use EXPLAIN to understand the plan
3. **Optimize** - Add indexes, rewrite queries
4. **Verify** - Confirm improvement with benchmarks
5. **Monitor** - Track performance over time

In [ ]:
# Cleanup
conn.close()
print("Database connection closed.")